In [1]:
import pandas as pd
import os
from get_file_from_email import *
from db_funcs import *
from normalize_funcs import *
import config
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
db_name = config.db_name

# Файл для создания справочника weborama_accounts - собрал руками и сохранил в эксель
# это нужно только для 1 загрузки, чтобы потом сверяться с таблице из БД и проверять изменения
file_name = 'weborama_accounts.xlsx'
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\38_x5_report_parser\weborama'

In [2]:
# при первом подключении заюираем справочник Источников из MySQL
# необходимо его пересоздать в MSSQL
# при повторных подключениях будем работать именно с MSSQL
table_name = 'source_types'
df_sources = get_mysql_full_dict_table(db_name, table_name)

df_sources = df_sources.drop(['created_at', 'updated_at'], axis=1)

Все ок. Подключились!
Данные загружены


In [3]:
# Справочник Источников
# создаем общий список названий полей и типов данных 
# этот список передаем в БД MSSQL для создания новой таблицы

db_vars_str = [
            'id smallint',
            'name nvarchar(100)',
            'ak_source_name nvarchar(100)',
            'sign nvarchar(100)',
            'utm_source_main nvarchar(100)',
            'source_engine nvarchar(100)',
            'utm_source_metrika nvarchar(100)']

In [4]:
# создаем пустую таблицу Справочников в БД
table_name = 'full_source_types'
createDBTable(db_name, table_name, db_vars_str)

Все ок. Подключились!
Пустая таблица full_source_types успешно создана в БД tenant_perekrestok_project


In [5]:
int_lst = ['id']
df_sources = normalize_columns_types(df_sources, int_lst)

In [6]:
# записываем справочник Источников в БД
table_name = 'full_source_types'
downloadTableToDB(db_name, table_name, df_sources)

Все ок. Подключились!
Скрипт запущен 2025-02-26 17:38:58.773063
Данные добавлены в БД: tenant_perekrestok_project, таблица: full_source_types
Скрипт отработал 2025-02-26 17:38:58.791067
Время выполнения задачи: 0:00:00.018004
Загрузка завершена. Данные успешно добавлены в БД: tenant_perekrestok_project
##################################################



In [7]:
table_name = 'accounts'
df_accs = get_mysql_full_dict_table(db_name, table_name)

df_accs = df_accs[['id', 'source_type_id', 'account_name', 'account_id', 'acc_id_flag']]
df_accs['weborama_account_name'] = 'x5_perekrestok'

Все ок. Подключились!
Данные загружены


In [8]:
# Общий Справочник Аккаунтов
# создаем общий список названий полей и типов данных 
# этот список передаем в БД MSSQL для создания новой таблицы

db_vars_str = [
            'id smallint',
            'source_type_id smallint',
            'account_name nvarchar(100)',
            'account_id smallint',
            'acc_id_flag nvarchar(3)',
            'weborama_account_name nvarchar(100)']

In [9]:
# создаем пустую таблицу Аккаунтов в БД
table_name = 'full_accounts_dict'
createDBTable(db_name, table_name, db_vars_str)

Все ок. Подключились!
Пустая таблица full_accounts_dict успешно создана в БД tenant_perekrestok_project


In [10]:
int_lst = ['id', 'source_type_id', 'account_id']
df_accs = normalize_columns_types(df_accs, int_lst)

In [11]:
# записываем справочник Аккаунтов в БД
table_name = 'full_accounts_dict'
downloadTableToDB(db_name, table_name, df_accs)

Все ок. Подключились!
Скрипт запущен 2025-02-26 17:39:16.601419
Данные добавлены в БД: tenant_perekrestok_project, таблица: full_accounts_dict
Скрипт отработал 2025-02-26 17:39:16.619418
Время выполнения задачи: 0:00:00.017999
Загрузка завершена. Данные успешно добавлены в БД: tenant_perekrestok_project
##################################################



In [12]:
# Справочник Аккаунтов Weborama
# создаем общий список названий полей и типов данных 
# этот список передаем в БД MSSQL для создания новой таблицы

db_vars_str = [
            'source_type_id smallint',
            'account_name nvarchar(100)',
            'account_id smallint']

In [13]:
# создаем пустую таблицу Аккаунтов в БД
table_name = 'weborama_accounts'
createDBTable(db_name, table_name, db_vars_str)

Все ок. Подключились!
Пустая таблица weborama_accounts успешно создана в БД tenant_perekrestok_project


In [15]:
df_weborama_accounts = pd.read_excel(os.path.join(file_path, file_name))
df_weborama_accounts = df_weborama_accounts[['source_type_id', 'account_name', 'account_id']]

In [16]:
int_lst = ['source_type_id', 'account_id']
df_weborama_accounts = normalize_columns_types(df_weborama_accounts, int_lst)
df_weborama_accounts = df_weborama_accounts.sort_values('source_type_id')

In [17]:
# записываем справочник Аккаунтов Weborama в БД
table_name = 'weborama_accounts'
downloadTableToDB(db_name, table_name, df_weborama_accounts)

Все ок. Подключились!
Скрипт запущен 2025-02-26 17:40:03.126520
Данные добавлены в БД: tenant_perekrestok_project, таблица: weborama_accounts
Скрипт отработал 2025-02-26 17:40:03.140519
Время выполнения задачи: 0:00:00.013999
Загрузка завершена. Данные успешно добавлены в БД: tenant_perekrestok_project
##################################################



In [18]:
# Добавим аккаунты Веборамы к таблице всех Аккаунтов
# Это тоже первая заливка, в дальнейшем будем добавлять только новые аккаунты
table_name = 'weborama_accounts'
df_weborama_accounts = get_mssql_table(db_name, table_name)

Все ок. Подключились!
Загрузка завершена успешно


In [19]:
table_name = 'full_accounts_dict'
df_accounts = get_mssql_table(db_name, table_name)

Все ок. Подключились!
Загрузка завершена успешно


In [20]:
max_account_id = df_accounts['id'].max()+1 # забираем максимальный ИД из справояника MSSQL
# создаем нумерацию новых аккаунтов, начиная с последней записи из общего справочника аккаунтов
ids_list = [i for i in range(max_account_id, len(df_weborama_accounts)+max_account_id)]
# добавляем новое поле к таблице аккаунтов из Веборамы
df_weborama_accounts['id'] = pd.Series(ids_list)
# добавляем ведущий ноль к ИД аккаунта
df_weborama_accounts['acc_id_flag'] = df_weborama_accounts['id'].apply(lambda x: '0' + str(x) if len(str(x))<2 else str(x))
# добавляем поле с названием аккаунта, чтобы сошлись таблицы 
df_weborama_accounts['weborama_account_name'] = df_weborama_accounts['account_name']
# создаем итоговый датаФрейм для записи с Общим справочником аккаунтов
df_union_accounts = pd.concat([df_accounts, df_weborama_accounts])

In [21]:
# Общий Справочник Аккаунтов
# создаем общий список названий полей и типов данных 
# этот список передаем в БД MSSQL для создания новой таблицы

db_vars_str = [
            'id smallint',
            'source_type_id smallint',
            'account_name nvarchar(100)',
            'account_id smallint',
            'acc_id_flag nvarchar(3)',
            'weborama_account_name nvarchar(100)']

In [22]:
# записываем новую таблицу в БД
table_name = 'full_accounts_dict'

createDBTable(db_name, table_name, db_vars_str, flag='drop')
downloadTableToDB(db_name, table_name, df_union_accounts)

Все ок. Подключились!
Все ок. Подключились!
Таблица: full_accounts_dict успешно удалена в БД: tenant_perekrestok_project
##########
Пустая таблица full_accounts_dict успешно создана в БД tenant_perekrestok_project
Все ок. Подключились!
Скрипт запущен 2025-02-26 17:40:24.017973
Данные добавлены в БД: tenant_perekrestok_project, таблица: full_accounts_dict
Скрипт отработал 2025-02-26 17:40:24.036973
Время выполнения задачи: 0:00:00.019000
Загрузка завершена. Данные успешно добавлены в БД: tenant_perekrestok_project
##################################################



In [ ]:
# Ежедневный отчет Веборама
# создаем общий список названий полей и типов данных 
# этот список передаем в БД MSSQL для создания новой таблицы

db_vars_str = [
            'date nvarchar(10)',
            'campaign_id smallint',
            'campaign_name nvarchar(100)',
            'site_offer_id smallint',
            'source nvarchar(100)',
            'insertion_id smallint',
            'insertion nvarchar(200)',
            'account_name nvarchar(100)',
            'impressions int',
            'clicks int',
            'reach_increment int',
            'video_views_25 int',
            'video_views_50 int',
            'video_views_75 int',
            'video_views_100 int',
            'viewable_views int',
            'type nvarchar(100)',
            'category nvarchar(100)',
            'product nvarchar(100)',
            'flight nvarchar(100)',
            'weborama_camp_name nvarchar(200)',
            'source_type_id smallint',
            'main_acc_id smallint',
            'weborama_key_camp nvarchar(200)'
]


In [ ]:
# пересоздаем пустую таблицу Справочников в БД
table_name = 'weborama_report_table'
createDBTable(db_name, table_name, db_vars_str)

# dropTable(db_name, table_name)